#### Imports

In [1]:
import os
import re
import skrf as rf
import pandas as pd

#### Paths

In [2]:
script_directory = os.getcwd()#obtem diretório atual
s2ps_path = os.path.join(script_directory,'..','..','oil_data','oil_data') #cria string caminho para todos os .s2p

#### Regex

In [3]:
regex = r'SAW_06_02_2024_OIL_10W40_(\d+)\.S2P'

In [4]:
def read_s2p_data(regex, path):
    dfs = []#lista para ocncatenar todos os dicts de cada medição 
    for filename in os.listdir(path):
        match = re.match(regex, filename)
        if match:
            #obtem informação do nome do arquivo
            oil_name = "10W40"
            measurement_number = int(match.group(1))
            
            #carrega arquivo
            path_to_oil = os.path.join(path, filename)
            s2p_file = rf.Network(path_to_oil)
            
            #extrai frequência e parâmetros S do objeto Network
            frequency = s2p_file.f
            s_parameters = s2p_file.s_db[:,1,0]
            phase = s2p_file.s_deg[:,1,0]
            
            #Dataframe com informações selecionadas
            individual_oil_dict = {
                'OIL NAME': oil_name,
                'MEASUREMENT NUMBER': measurement_number,
                'FREQUENCY (MHZ)':frequency/1000000,
                'S21 (db)' : s_parameters,
                'PHASE (DEG)': phase,
            }
            df = pd.DataFrame.from_dict(individual_oil_dict)
            df = df.nlargest(3, 'S21 (db)')
            dfs.append(df)
            
    dataframe = pd.concat(dfs, ignore_index=True)
    return dataframe

#### Read Dataframes

In [5]:
df_synthetic = read_s2p_data(regex, s2ps_path)

In [6]:
#assegura somente óleo de 10W40 lido
for name in df_synthetic["OIL NAME"]:
    if name != "10W40":
        print("Diferente de 10W40")

In [7]:
#cria .csv caso precise no futuro.
path_save_csv = os.path.join(script_directory,'..','..', 'oil_data','oil_data','CSVs')
if not os.path.exists(path_save_csv):
   os.makedirs(path_save_csv)
df_synthetic.to_csv(path_save_csv + '/10W40_measurements.csv')